<a href="https://colab.research.google.com/github/amedyukhina/AgrifieldNet/blob/main/agrinet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AgrifieldNet India Competition

## Install and import dependencies


In [1]:
%%writefile requirements.txt

pandas==1.3.5
scikit-learn==1.0.2
tqdm==4.64.0
rasterio==1.2.0
numpy==1.21.6
radiant_mlhub==0.4.1
torch==1.12.1
torchvision==0.13.1
scikit-image==0.18.3
matplotlib==3.2.2
pillow==7.1.2
monai==0.9.1
tensorboard==2.8.0

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import json
import getpass
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
from radiant_mlhub import Dataset
import tarfile
import shutil
import numpy as np
import pandas as pd
import torch
import torch.utils.data
from torch import nn
from skimage import io
from torchvision import transforms as torch_transforms
import pylab as plt
from PIL import Image
from skimage.segmentation import mark_boundaries
from monai.networks.layers import Norm
from monai.networks.nets import UNet
from monai.losses import DiceLoss, DiceCELoss, MaskedDiceLoss
from monai.metrics import DiceMetric
# from torch.utils.tensorboard import SummaryWriter

## Specify global parameters

In [4]:
data_dir = 'data'
random_seed = 42

train_df_path = 'train_data.csv'

BAND_NAMES = ['B01', 'B02', 'B03', 'B04','B05', 'B06', 'B07', 'B08','B8A', 'B09', 'B11', 'B12']

In [5]:
np.random.seed(random_seed)

## Download data from MLHUB


In [6]:
collection_name = 'ref_agrifieldnet_competition_v1'

source_collection = f'{collection_name}_source'
train_label_collection = f'{collection_name}_labels_train'
test_label_collection = f'{collection_name}_labels_test'

In [7]:
if not os.path.exists(data_dir):
  os.environ['MLHUB_API_KEY'] =  getpass.getpass(prompt="MLHub API Key: ")

  dataset = Dataset.fetch(collection_name)
  dataset.download(output_dir=data_dir)
  for fn in os.listdir(data_dir):
    with tarfile.open(os.path.join(data_dir, fn)) as f:
      f.extractall(data_dir + '/') 
    os.remove(os.path.join(data_dir, fn))
else:
  print("Dataset already exists")

Dataset already exists


## Prepare Training data


### Create training dataframe

In [8]:
train_paths = os.listdir(os.path.join(data_dir, train_label_collection))
train_ids = [fn.split('_')[-1] for fn in train_paths if 'labels_train' in fn]

field_paths = [f'{data_dir}/{train_label_collection}/{train_label_collection}_{i}/field_ids.tif' 
               for i in train_ids]
label_paths = [f'{data_dir}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' 
               for i in train_ids]
source_paths = [f'{data_dir}/{source_collection}/{source_collection}_{i}/' 
               for i in train_ids]

In [9]:
train_data = pd.DataFrame(np.array([train_ids, field_paths, label_paths, source_paths]).transpose(), 
                          columns=['folder_id', 'field_path', 'label_path', 'source_path'])
train_data.head()

,folder_id,field_path,label_path,source_path
0,8a219,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
1,b2361,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
2,ae507,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
3,39ea6,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
4,001c1,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...



## Extract field-crop data

In [10]:
def extract_field_crop_data(data):
  field_ids = []
  crop_type = []
  field_area = []
  field_max_dim = []
  field_center_x = []
  field_center_y = []
  label_paths = []
  field_paths = []
  source_paths = []

  for i in tqdm(range(len(data))):
      with rasterio.open(data['field_path'].iloc[i]) as src:
          field_data = src.read()[0]
      with rasterio.open(data['label_path'].iloc[i]) as src:
          crop_data = src.read()[0]

      for field_id in np.unique(field_data)[1:]:
          ind = np.where(field_data == field_id)
          field_ids.append(field_id)
          crop_type.append(np.unique(crop_data[ind])[-1])
          field_area.append(len(ind[0]))
          field_max_dim.append(np.max(np.array(ind).max(1) - np.array(ind).min(1) + 1))
          field_center_y.append(np.mean(ind[0]))
          field_center_x.append(np.mean(ind[1]))
          label_paths.append(data['label_path'].iloc[i])
          field_paths.append(data['field_path'].iloc[i])
          source_paths.append(data['source_path'].iloc[i])

  df = pd.DataFrame(np.array([field_ids, crop_type, field_area, 
                              field_max_dim, field_center_x, 
                              field_center_y]).transpose(),
                    columns=['field_id', 'crop_type', 'field_area', 
                            'field_max_dim', 'center_x', 'center_y'])
  df['label_path'] = label_paths
  df['field_path'] = field_paths
  df['source_path'] = source_paths
  return df

In [ ]:
df = extract_field_crop_data(train_data)

 73%|███████▎  | 855/1165 [00:11<00:03, 79.56it/s]

In [ ]:
maxdim = df['field_max_dim'].max()
maxdim

In [ ]:
size = 64

In [ ]:
mean_area_fraction = np.mean(df['field_area']) / 256**2
mean_area_fraction

In [ ]:
df = df.sort_values(['crop_type', 'field_id']).reset_index(drop=True)
df['count'] = 1

In [ ]:
df.groupby('crop_type').sum()

### Split the data into train and validation

In [ ]:
val_fraction = 0.2

In [ ]:
df_train = []
df_val = []
for crop in df['crop_type'].unique():
    cur_df = df[df['crop_type'] == crop].reset_index(drop=True)
    unique_field_ids = cur_df['field_id'].unique()
    ind = np.arange(len(unique_field_ids))
    np.random.shuffle(ind)
    n_val = int(round(val_fraction * len(ind)))
    df_val.append(cur_df[cur_df['field_id'].isin(unique_field_ids[ind[:n_val]])])
    df_train.append(cur_df[cur_df['field_id'].isin(unique_field_ids[ind[n_val:]])])
df_train = pd.concat(df_train, ignore_index=True)
df_val = pd.concat(df_val, ignore_index=True)

In [ ]:
n_fields = np.array(df_train.groupby('crop_type').sum()['count'])
weights = np.max(n_fields) / n_fields
crop_labels = np.unique(df_train['crop_type'])

In [ ]:
crop_labels, weights

In [ ]:
loss_weights = np.zeros(int(crop_labels.max()) + 1)
loss_weights[0] = mean_area_fraction
loss_weights[list(np.int_(crop_labels))] = weights
loss_weights

## Extract labels

In [ ]:
with open(f'{data_dir}/{train_label_collection}/{train_label_collection}'
          f'_{train_data.iloc[0]["folder_id"]}/stac.json') as ll:
    label_json = json.load(ll)

In [ ]:
crop_dict = {asset.get('values')[0]:asset.get('summary') 
  for asset in label_json['assets']['raster_labels']['file:values']}
crop_dict

## Specify Data Loading pipeline

### Specify parameters

In [ ]:
batch_size = 16

### Specify the Dataset

In [ ]:
class AgriDataset(torch.utils.data.Dataset):

  def __init__(self, df, transforms=None, 
               band_names=BAND_NAMES, size=64):
    self.df = df
    self.transforms = transforms
    self.band_names = band_names
    self.size = size

  def __getitem__(self, index):
    imgs = []
    for band in self.band_names:
      source_fn = rf"{self.df.iloc[index]['source_path']}{band}.tif"
      with rasterio.open(source_fn) as src:
        imgs.append(src.read()[0].astype(np.float64)/255.)
    with rasterio.open(self.df.iloc[index]['label_path']) as src:
      labels = src.read()[0].astype(np.int64)

    labels = torch.tensor(labels).unsqueeze(0)
    imgs = torch.tensor(np.array(imgs))
    
    hs = int(self.size/2)
    ind = [self.df.iloc[index]['center_y'], self.df.iloc[index]['center_x']]
    ind = np.max(np.array([[hs, hs], ind]), axis=0)
    ind = np.int_(np.min(np.array([np.array(imgs.shape[-2:])-hs-1, ind]), axis=0))
    labels = labels[:, ind[0]-hs:ind[0]+hs, ind[1]-hs:ind[1]+hs]
    imgs = imgs[:, ind[0]-hs:ind[0]+hs, ind[1]-hs:ind[1]+hs]
      
    if self.transforms:
      seed = np.random.randint(np.iinfo('int32').max)
      torch.manual_seed(seed)
      labels = self.transforms(labels)
      torch.manual_seed(seed)
      imgs = self.transforms(imgs)

    return imgs.float(), labels
    

  def __len__(self):
    return len(self.df)

### Specify transforms

In [ ]:
train_transforms = torch_transforms.Compose([
        torch_transforms.RandomHorizontalFlip(),
        torch_transforms.RandomVerticalFlip(),
        torch_transforms.RandomRotation(degrees=30),
    ])

validation_transforms = torch_transforms.Compose([
        torch_transforms.RandomHorizontalFlip(),
        torch_transforms.RandomVerticalFlip(),
    ])

### Test the loaders

In [ ]:
np.random.seed(random_seed)
dl_train0 = torch.utils.data.DataLoader(
    AgriDataset(df_train, size=size,
                # transforms=train_transforms
                ), 
    shuffle=False, batch_size=batch_size, num_workers=2
    )
dl_train = torch.utils.data.DataLoader(
    AgriDataset(df_train, size=size,
                transforms=train_transforms
                ), 
    shuffle=False, batch_size=batch_size, num_workers=2
    )
dl_val = torch.utils.data.DataLoader(
    AgriDataset(df_val, size=size,
                transforms=validation_transforms
                ), 
    shuffle=False, batch_size=batch_size, num_workers=2
    )

In [ ]:
imgs, labels = next(iter(dl_train))
imgs0, labels0 = next(iter(dl_train0))

In [ ]:
imgs.shape, labels.shape

In [ ]:
ind = 0
s = 3

In [ ]:
plt.figure(figsize=(s,s))
io.imshow(labels[ind][0].numpy())

In [ ]:
plt.figure(figsize=(s,s))
io.imshow(labels0[ind][0].numpy())

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(s*4, s*3))
for ax, img in zip(axes.ravel(), imgs[ind]):
  plt.sca(ax)
  im = img.numpy()
  im = im / np.max(im)
  io.imshow(mark_boundaries(im, labels[ind][0].numpy(), 
                            mode='outer'))

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(s*4, s*3))
for ax, img in zip(axes.ravel(), imgs0[ind]):
  plt.sca(ax)
  im = img.numpy()
  im = im / np.max(im)
  io.imshow(mark_boundaries(im, labels0[ind][0].numpy(), 
                            mode='outer'))

### Specify the loaders

In [ ]:
np.random.seed(random_seed)

dl_train = torch.utils.data.DataLoader(
    AgriDataset(df_train, size=size,
                transforms=train_transforms
                ), 
    shuffle=True, batch_size=batch_size, num_workers=2
    )
dl_val = torch.utils.data.DataLoader(
    AgriDataset(df_val, size=size,
                transforms=validation_transforms
                ), 
    shuffle=False, batch_size=batch_size, num_workers=2
    )

## Specify the training pipeline

### Parameters

In [ ]:
out_channels = int(np.max(crop_labels) + 1)
n_channels = (32, 64, 128, 256, 512, 512)
strides = (1, 2, 1, 2, 2)
num_res_units = 3
lr = 0.0001
weight_decay = 0.0005
factor = 0.1
patience = 5
epochs = 100

### Model and loss

In [ ]:
net = UNet(
    spatial_dims=2,
    in_channels=len(BAND_NAMES),
    out_channels=out_channels,
    channels=n_channels,
    strides=strides,
    num_res_units=num_res_units,
    norm=Norm.BATCH,
).cuda()

In [ ]:
# class MaskedDiceCELoss(nn.Module):

#   def __init__(self, include_background=False, to_onehot_y=True, softmax=True,
#                dice_ce_weight=1, dice_mask_weight=1):
    
#     super().__init__()
#     self.diceCELoss = DiceCELoss(include_background=include_background, 
#                                  to_onehot_y=to_onehot_y, 
#                                  softmax=softmax)
#     self.diceMasked = MaskedDiceLoss(include_background=include_background, 
#                                        to_onehot_y=to_onehot_y, 
#                                        softmax=softmax)
#     self.dice_ce_weight = dice_ce_weight
#     self.dice_mask_weight = dice_mask_weight

#   def forward(self, input, target, mask):
#     loss1 = self.diceCELoss(input, target)*self.dice_ce_weight 
#     loss2 = self.diceMasked(input, target, mask)*self.dice_mask_weight
#     return loss1 + loss2

In [ ]:
# loss_fn = MaskedDiceCELoss(include_background=False, 
#                            to_onehot_y=True, softmax=True).cuda()
loss_fn = DiceCELoss(include_background=False,
                     to_onehot_y=True, softmax=True,
                     ce_weight=torch.tensor(loss_weights).float()).cuda()
metric = DiceMetric(include_background=False, reduction="mean")

### Optimizer and scheduler

In [ ]:
optimizer = torch.optim.AdamW(
    params=[{"params": [p for p in net.parameters() if p.requires_grad]}],
            lr=lr, weight_decay=weight_decay
            )

In [ ]:
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=factor, patience=patience
    )

### Training step

In [ ]:
def train_epoch(net, loss_fn, dl_train, optimizer):
  net.train()
  loss_fn.train()
  epoch_loss = 0
  step = 0
  for imgs, targets in tqdm(dl_train):
    step += 1
    optimizer.zero_grad()
    outputs = net(imgs.cuda())
    losses = loss_fn(outputs, targets.cuda())
    losses.backward()
    optimizer.step()
    epoch_loss += losses.item()
  epoch_loss /= step
  return epoch_loss

### Validation step

In [ ]:
def validate_epoch(net, loss_fn, dl_val, metric):
  net.eval()
  loss_fn.eval()
  epoch_loss = 0
  step = 0

  with torch.no_grad():
    for imgs, targets in tqdm(dl_val):
      step += 1
      outputs = net(imgs.cuda())
      losses = loss_fn(outputs, targets.cuda())
      epoch_loss += losses.item()
      metric(outputs.argmax(1).unsqueeze(1), targets.cuda())
  epoch_loss /= step
  return epoch_loss, metric

## Train the model

In [ ]:
%%time
best_metric = 0
train_losses = []
val_losses = []
val_metrics = []
for epoch in range(epochs):
    # training pass
    train_loss = train_epoch(net, loss_fn, dl_train, optimizer)

    # validation pass
    val_loss, metric = validate_epoch(net, loss_fn, dl_val, metric)
    val_metric = metric.aggregate().item()
    metric.reset()

    # update learning rate
    lr_scheduler.step(val_loss)

    # output
    print(f"epoch {epoch + 1} training loss: {train_loss:.4f}; "
          f"validation loss: {val_loss:.4f}; Dice metric: {val_metric:.4f}; "
          f"lr: {optimizer.param_groups[0]['lr']}")
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_metrics.append(val_metric)
    if val_metric >= best_metric:
      best_metric = val_metric
      torch.save(net.state_dict(), 'model.pth')
      print('saving best model')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses,'r')
plt.plot(val_losses, 'b')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(val_metrics,'r')

## Set up the loaders and model for evaluation

In [ ]:
class AgriDatasetTest(torch.utils.data.Dataset):

  def __init__(self, df, band_names=BAND_NAMES, size=64):
    self.band_names = band_names
    self.df = df
    self.size = size

  def __getitem__(self, index):
    imgs = []
    for band in self.band_names:
      source_fn = rf"{self.df.iloc[index]['source_path']}{band}.tif"
      with rasterio.open(source_fn) as src:
        imgs.append(src.read()[0].astype(np.float64)/255.)

    with rasterio.open(self.df.iloc[index]['field_path']) as src:
      labels = src.read()[0].astype(np.int64)

    labels = torch.tensor(labels)
    imgs = torch.tensor(np.array(imgs))

    hs = int(self.size/2)
    ind = [self.df.iloc[index]['center_y'], self.df.iloc[index]['center_x']]
    ind = np.max(np.array([[hs, hs], ind]), axis=0)
    ind = np.int_(np.min(np.array([np.array(imgs.shape[-2:])-hs-1, ind]), axis=0))

    imgs = imgs[:, ind[0]-hs:ind[0]+hs, ind[1]-hs:ind[1]+hs]
    labels = labels[ind[0]-hs:ind[0]+hs, ind[1]-hs:ind[1]+hs]

    if 'label_path' in self.df.columns and os.path.exists(self.df.iloc[index]['label_path']):
      with rasterio.open(self.df.iloc[index]['label_path']) as src:
        gt = src.read()[0].astype(np.int64)
        gt = torch.tensor(gt)[ind[0]-hs:ind[0]+hs, ind[1]-hs:ind[1]+hs]
    else:
      gt = labels


    return imgs.float(), labels, gt

  def __len__(self):
    return len(self.df)

In [ ]:
dl_test = torch.utils.data.DataLoader(
    AgriDatasetTest(df_val),
    shuffle=False, batch_size=batch_size, num_workers=2
    )

In [ ]:
net.load_state_dict(torch.load('model.pth'))
net.eval();

## Evaluate the model on the validation set

In [ ]:
def evaluate(dl_test, net):
  field_data = dict()
  gt_labels = dict()
  with torch.no_grad():
    for imgs, labels, gt in tqdm(dl_test):
      output = net(imgs.cuda()).cpu()
      output = output.numpy()
      labels = labels.numpy()
      output = np.moveaxis(output, 1, -1)

      llist = np.unique(labels)[1:]
      for lb in llist:
        ind = np.where(labels == lb)
        data = output[ind]
        if lb in field_data.keys():
          field_data[lb] = np.concatenate([field_data[lb], data], axis=0)
        else:
          field_data[lb] = data

        data = gt[ind]
        if lb in gt_labels.keys():
          gt_labels[lb] = np.concatenate([gt_labels[lb], data], axis=0)
        else:
          gt_labels[lb] = data
        
  field_ids = []
  probs = []
  gt = []
  for key in field_data.keys():
    field_ids.append(key)
    prob = field_data[key].mean(0)[list(np.int_(crop_labels))]
    prob = prob - np.min(prob)
    prob = prob / np.sum(prob)
    probs.append(prob)
    gt.append(gt_labels[key][0].item())
  probs = np.array(probs)
  return probs, field_ids, gt

In [ ]:
probs, field_ids, gt = evaluate(dl_test, net)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(gt, np.array(crop_labels)[np.argmax(probs, axis=1)]))

In [ ]:
def to_one_hot(x):
    y = np.zeros((x.size, x.max() + 1))
    y[np.arange(x.size), x] = 1
    return y

def cross_entropy(predictions, targets):
    return -np.sum(np.sum(np.log(predictions + 10**(-10)) * targets, axis=1))/len(targets)


In [ ]:
targets = to_one_hot(np.array(gt))
targets = targets[:, np.where(targets.sum(axis=0) > 0)[0]]

In [ ]:
cross_entropy(probs, targets)

## Prepare the test dataset

In [ ]:
test_paths = os.listdir(os.path.join(data_dir, test_label_collection))
test_ids = [fn.split('_')[-1] for fn in test_paths if 'labels_test' in fn]

field_paths = [f'{data_dir}/{test_label_collection}/{test_label_collection}_{i}/field_ids.tif' 
               for i in test_ids]
label_paths = [f'{data_dir}/{test_label_collection}/{test_label_collection}_{i}/raster_labels.tif' 
               for i in test_ids]
source_paths = [f'{data_dir}/{source_collection}/{source_collection}_{i}/' 
               for i in test_ids]

In [ ]:
test_data = pd.DataFrame(np.array([test_ids, field_paths, label_paths, source_paths]).transpose(), 
                          columns=['folder_id', 'field_path', 'label_path', 'source_path'])
test_data.head()

In [ ]:
def extract_field_crop_data_test(data):
  field_ids = []
  field_center_x = []
  field_center_y = []
  field_paths = []
  source_paths = []

  for i in tqdm(range(len(data))):
      with rasterio.open(data['field_path'].iloc[i]) as src:
          field_data = src.read()[0]

      for field_id in np.unique(field_data)[1:]:
          ind = np.where(field_data == field_id)
          field_ids.append(field_id)
          field_center_y.append(np.mean(ind[0]))
          field_center_x.append(np.mean(ind[1]))
          field_paths.append(data['field_path'].iloc[i])
          source_paths.append(data['source_path'].iloc[i])

  df = pd.DataFrame(np.array([field_ids, field_center_x, 
                              field_center_y]).transpose(),
                    columns=['field_id', 'center_x', 'center_y'])
  df['field_path'] = field_paths
  df['source_path'] = source_paths
  return df

In [ ]:
test_df = extract_field_crop_data_test(test_data)

## Evaluate the model on the test set

In [ ]:
dl_test = torch.utils.data.DataLoader(
    AgriDatasetTest(test_df), shuffle=False, batch_size=batch_size, num_workers=2
    )

In [ ]:
probs, field_ids, gt = evaluate(dl_test, net)

In [ ]:
def labeler(labeled):
    crop_label = np.array([crop_dict.get(f'{int(i)}') for i in labeled])
    return crop_label

In [ ]:
crop_columns = [crop_dict[key] for key in crop_labels]

output_df  = pd.DataFrame(columns=['field_id'] + crop_columns)

output_df['field_id'] = field_ids

output_df[crop_columns] = probs 
output_df.to_csv('submission.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission.csv')